In [1]:
pip install turicreate

     |████████████████████████████████| 91.2MB 393kB/s  eta 0:00:01     |█████████████████▌              | 49.9MB 37.9MB/s eta 0:00:02
     |████████████████████████████████| 17.3MB 27.6MB/s eta 0:00:01     |███████████████████▉            | 10.8MB 27.6MB/s eta 0:00:01
     |████████████████████████████████| 3.4MB 26.5MB/s eta 0:00:01
     |████████████████████████████████| 421.8MB 87kB/s s eta 0:00:01    |█▉                              | 24.4MB 19.9MB/s eta 0:00:21     |███                             | 38.4MB 27.0MB/s eta 0:00:15     |████▊                           | 61.7MB 27.0MB/s eta 0:00:14     |████████                        | 104.1MB 38.8MB/s eta 0:00:09��████▋                     | 140.5MB 34.9MB/s eta 0:00:09��███████████████████▌    | 362.6MB 33.8MB/s eta 0:00:02��███▋    | 363.7MB 33.8MB/s eta 0:00:02ta 0:00:01��████████████████| 420.6MB 23.1MB/s eta 0:00:01
     |████████████████████████████████| 327kB 31.5MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 3

In [2]:
import turicreate as tc

In [3]:
sales = tc.SFrame('home_data.sframe')

In [4]:
import numpy as np

In [13]:
def get_numpy_data(data_sframe,features,output):
    data_sframe['constant']=1
    features=['constant']+features
    features_sframe = data_sframe[features]
    feature_matrix=features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    return(feature_matrix,output_array)

In [6]:
def predict_output(feature_matrix,weights):
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [7]:
def feature_derivatives(errors,feature):
    derivative = 2*(np.dot(errors,feature))
    return(derivative)

In [17]:
def regression_gradient_descent(feature_matrix,output,initial_weights,step_size,tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
            predictions = predict_output(feature_matrix,weights)
            errors = predictions-output
            gradient_sum_squares = 0
            for i in range(len(weights)):
                derivative = feature_derivatives(errors,feature_matrix[:,i])
                gradient_sum_squares = gradient_sum_squares+(derivative**2)
                weights[i]=weights[i]-(step_size*derivative)
            gradient_magnitude = sqrt(gradient_sum_squares)
            if gradient_magnitude < tolerance:
                converged = True
    return(weights)

In [6]:
train_data,test_data = sales.random_split(.8,seed=0)

In [14]:
simple_features=['sqft_living']
my_output = 'price'
simple_feature_matrix,output=get_numpy_data(train_data,simple_features,my_output)
initial_weights = np.array([-47000.,1.])
step_size = 7e-12
tolerance = 2.5e7

In [18]:
simple_weights = regression_gradient_descent(simple_feature_matrix,output,initial_weights,step_size,tolerance)

In [16]:
from math import sqrt 

In [19]:
simple_weights

array([-46999.88716555,    281.91211912])

In [20]:
test_simple_features=['sqft_living']
test_my_output = 'price'
test_simple_feature_matrix,test_output=get_numpy_data(test_data,test_simple_features,test_my_output)

In [22]:
test_weight = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print(test_weight)

[-46999.88716555    281.91211912]


In [23]:
test_predictions = predict_output(test_simple_feature_matrix, test_weight)
print(test_predictions)

[356134.44317093 784640.86422788 435069.83652353 ... 663418.65300782
 604217.10799338 240550.4743332 ]


In [27]:
test_residuals =(test_output - test_predictions)
test_RSS = (test_residuals * test_residuals).sum()
print(test_RSS)

275400047593155.94


In [28]:
model_features = ['sqft_living', 'sqft_living15']  
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [29]:
weight_2 = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print(weight_2)

[-9.99999688e+04  2.45072603e+02  6.52795277e+01]


In [31]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
test_predictions_2 = predict_output(test_feature_matrix, weight_2)
print(test_predictions_2)

[366651.41203656 762662.39786164 386312.09499712 ... 682087.39928241
 585579.27865729 216559.20396617]


In [32]:
print(test_predictions_2[0])
print(test_data['price'][0])

366651.4120365591
310000.0


In [33]:
test_residuals_2 = test_output - test_predictions_2
test_RSS_2 = (test_residuals_2**2).sum()
print(test_RSS_2)

270263446465244.06


In [7]:
test_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901.0,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000.0,2.0,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697.0,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980.0,2.0,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000.0,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553.0,2.0,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186.0,1.0,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495.0,2.0,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400.0,1.0,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807.0,1.5,0
